This notebook is used for creating the plots in the paper.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import LogNorm
from matplotlib.ticker import (
    FixedLocator,
    FuncFormatter,
    StrMethodFormatter,
)
import numpy as np

from covvvr import CVIntegrator
from covvvr.functions import *

from scripts.constants import DATA_DIR


---

# Variance Percentage Reduction for 2 Control Variates 

## Constants

In [ ]:
func = "2d_camel"
iters = 50
evts = 5000
N = 100

file_path = DATA_DIR / f"2cv_{func}_{iters}x{evts}_avg{N}.npz"


In [ ]:
# Ratio of main plot to the side plots
ratio = [11, 2]
# Fontsize of the labels
label_fs = 15
# Linewidth
lw = 3
# Colormap for the colorbar/heatmap
cmap = "OrRd"
# Colors to use for the variance and correlation coefficient lines
line_cmap = "magma"
colors = {
    "corrcoefs": colormaps[line_cmap](0.3),
    "variances": colormaps[line_cmap](0.7),
}
# Use a log colorbar?
do_log = False
# For side axes from 0 to 1
major_intr = 0.5
minor_intr = 0.25
major_locator = np.arange(0, 1 + major_intr, major_intr)
minor_locator = np.arange(0, 1 + minor_intr, minor_intr)
# For side axes from 1 to iters - 1
num_ticks = 5
iters_major_locator = np.concatenate(
    ([1], np.arange(0, iters, iters // num_ticks)[1:], [iters - 1])
)


## Load in Data

In [ ]:
data = np.load(file_path)
variances = np.mean(data["variances"], axis=0)
vars_norm = variances / np.max(variances)
corrcoefs = np.mean(data["corrcoefs"], axis=0)
perc_diff = 100 * np.mean(data["perc_diff"], axis=0)


## Plot Heatmap

In [ ]:
fig, ax = plt.subplots(
    2,
    2,
    figsize=(10, 10),
    gridspec_kw={"height_ratios": ratio, "width_ratios": ratio[::-1]},
)

# Plot the Variances
ax[0, 0].plot(
    vars_norm,
    range(1, iters),
    lw=lw,
    c=colors["variances"],
    label="Normalized\nVariance",
)
# Plot the correlation coefficients
ax[0, 0].plot(
    corrcoefs,
    range(1, iters),
    lw=lw,
    c=colors["corrcoefs"],
    label="Correlation\nCoefficient",
)
ax[0, 0].set_xlim(-0.05, 1.05)
ax[0, 0].set_ylim(1, iters - 1)
ax[0, 0].set_ylabel("Control Variate Iteration", fontsize=label_fs)
# Make sure all the ticks are looking good
ax[0, 0].yaxis.set_major_locator(FixedLocator(iters_major_locator))
ax[0, 0].xaxis.set_minor_locator(FixedLocator(minor_locator))
ax[0, 0].xaxis.set_major_locator(FixedLocator(major_locator))
# No decimal points for intergers
ax[0, 0].xaxis.set_major_formatter(
    FuncFormatter(lambda s, pos: f"{s:.0f}" if s == int(s) else f"{s:.1f}")
)
ax[0, 0].xaxis.set_minor_locator(FixedLocator(minor_locator))

# Either use a log colorbar or start the colorbar at zero
kwargs = {"norm": LogNorm()} if do_log else {"vmin": 0}
h = ax[0, 1].imshow(perc_diff, cmap=cmap, **kwargs)
ax[0, 1].invert_yaxis()
ax[0, 1].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)

ax[1, 0].set_visible(False)

# Same as with ax[0, 0] but the axes are flipped
ax[1, 1].plot(range(1, iters), vars_norm, lw=lw, c=colors["variances"])
ax[1, 1].plot(range(1, iters), corrcoefs, lw=lw, c=colors["corrcoefs"])
ax[1, 1].set_ylim(-0.05, 1.05)
ax[1, 1].set_xlim(1, iters - 1)
ax[1, 1].set_xlabel("Control Variate Iteration", fontsize=label_fs)
ax[1, 1].xaxis.set_major_locator(FixedLocator(iters_major_locator))
ax[1, 1].yaxis.set_minor_locator(FixedLocator(minor_locator))
ax[1, 1].yaxis.set_major_locator(FixedLocator(major_locator))
# Remove the '1' on the axis because it overlaps with the 1 from ax[0, 0]
ax[1, 1].yaxis.set_major_locator(FixedLocator(major_locator[:-1]))
ax[1, 1].yaxis.set_major_formatter(
    FuncFormatter(lambda s, pos: f"{s:.0f}" if s == int(s) else f"{s:.1f}")
)
ax[1, 1].yaxis.set_minor_locator(FixedLocator(minor_locator))

# Colorbar
cax = plt.axes([0.95, 0.1, 0.05, 0.78])
cbar = plt.colorbar(h, cax=cax, format=StrMethodFormatter("{x:>5.2f}%"))
cbar.set_label("Variance Percentage Reduction", labelpad=20, fontsize=label_fs)

fig.legend(loc=(0.04, 0.07))

# Squish 'em together
plt.subplots_adjust(wspace=0, hspace=0)


---

# Correlation of $f/p$ and $g/p$ 

## Constants

In [ ]:
# Parameters for the heatmap
label_fs = 17
cmap = "gist_yarg"

# Parameters for the lines
lw = 2
alpha = 0.4
color = "#2e282a"


## Do Integral

In [ ]:
# A 96D polynomial in this case
func = NPolynomial(96)
cvi = CVIntegrator(func, evals=10000, tot_iters=100, cv_iters=5, memory="large")
cvi.integrate()


## Plot it

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
hist = ax.hist2d(x=cvi.weight_value, y=cvi.cv_values[0], bins=(100, 250), cmap=cmap)
ax.set_ylabel("$g(x)/p(x)$", fontsize=label_fs)
ax.set_xlabel("$f(x)/p(x)$", fontsize=label_fs)

ax.tick_params(axis="both", which="major", labelsize=12)
textbox = f"$E[g/p]={np.mean(cvi.cv_values[0]):.5f}$"
textbox += (
    f"\n$\\rho(f/p,g/p)={np.corrcoef(cvi.weight_value, cvi.cv_values[0])[0, 1]:.2f}$"
)
bbox = dict(boxstyle="round, pad=1", fc="wheat", ec="black", alpha=0.5)
ax.annotate(
    textbox,
    xy=(0.6, 0.83),
    xycoords="axes fraction",
    fontsize=15,
    bbox=bbox,
)

# "perfect" values
ax.axhline(1, c=color, alpha=alpha, lw=lw)
ax.axvline(func.true_value, c=color, alpha=alpha, lw=lw)

ax.set_title(func.name, pad=15, fontsize=18)
